# New filter on summary

In [2]:
import polars as pl
df = pl.read_csv('df_clean_contsum.csv')
df.head()

sumary,content_cleaned
str,str
"""El consejo par…","""Para lograr un…"
"""Cómo gastar el…","""Dólar blue hoy…"
"""Bienvenido a E…","""Estimado lecto…"
"""La violencia v…","""La violencia e…"
"""Tres detenidos…","""27°SAN LUIS - …"


In [3]:
summaries = df['sumary'].to_list()
summaries[0]


'El consejo para ahorrar energía y agua al utilizar el lavarropa. La Federación Argentina de Cooperativas de Electricidad y Otros Servicios Públicos (FACE) debe dejar una serie de consejos para alcanzar la eficiencia energética'

In [4]:
keywords = pl.read_csv('/home/sebacastillo/willow/data/topics.csv')
keywords = keywords['keywords'].to_list()
keywords

['narcotráfico',
 'drogas',
 'cocaína',
 'marihuana',
 'heroína',
 'anfetaminas',
 'éxtasis',
 'traficante',
 'narcos',
 'narco',
 'estupefacientes',
 'incautación',
 'dealer',
 'mula',
 'clandestino',
 'cargamento']

In [5]:
import re
from nltk.stem.snowball import SnowballStemmer
from gensim.models.keyedvectors import KeyedVectors
from bs4 import BeautifulSoup
import string
import spacy
import numpy as np

# Stopword and stemer
stemmer = SnowballStemmer("spanish")
nlp = spacy.load("es_core_news_sm")
spanish_stopwords_spacy = spacy.lang.es.stop_words.STOP_WORDS

In [6]:
def compute_similarity(string, keywords, wordvec):
    """
    Compute the average similarity score between a list of topic words and all words in a string.
    Returns:
    - float: The average similarity score.
    """
    text = string.lower()
    text = re.sub(r"https?://", "", text)  # remove http/https
    text = re.sub(r"[\W_]+", " ", text)  # replace non-alphanumeric characters with space
    text = re.sub(r"[0-9]", " ", text)  # replace numeric characters with space
    string_words = [word for word in text.split() if word not in spanish_stopwords_spacy]

    total_scores = []
    for keyword in keywords:
        # Compute cosine similarity scores
        scores = []
        for word in string_words:
            if word in wordvec:
                similarity = wordvec.similarity(word, keyword)
                scores.append(similarity)

        max_scores = np.max(scores) if scores else 0.0
        total_scores.append(max_scores)

    # Return average score
    return sum(total_scores) / len(total_scores) if total_scores else 0.0


In [7]:
def load_embeddings(path="models/wiki.es.vec", limit=200000):
    """
    Load the word embeddings from the specified path.
    Args:
    - path (str): Path to the embeddings model.
    - limit (int): Limit the number of word vectors loaded.

    Returns:
    - KeyedVectors: Loaded word vectors.
    """
    return KeyedVectors.load_word2vec_format(path, limit=limit)

In [8]:
def filter_articles_with_similarity(
    strings, keywords, wordvectors, similarity_treshold
):
    
    s_second_match = []
    s_second_match_score = []

    for string in strings:

        similarity_score = compute_similarity(string, keywords, wordvectors)
        
        if similarity_score > similarity_treshold:
            s_second_match.append(string)
            s_second_match_score.append(similarity_score)

    return s_second_match, s_second_match_score

In [9]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = load_embeddings(path="/home/sebacastillo/willow/models/wiki.es.vec", limit=200000)

In [ ]:

sumary_match, summary_match_score = filter_articles_with_similarity(
                summaries, keywords, word_vectors, 0.2
            )


## Los sumarios Detectados

In [14]:
for sum, score in zip(sumary_match, summary_match_score):
    if score > 0.5:
        print(f'SCORE: {score} - sumary: {sum}')

SCORE: 0.5583932464942336 - sumary: Un 'punto de venta' de drogas en Posadas. La pareja, dueña del inmueble, terminó detenida por disposición de la Justicia Federal
SCORE: 0.6349147893488407 - sumary: Puerto Libertad incauta una tonelada de marihuana y un kilo de cocaína. La cadena del narcotráfico opera en Misiones
SCORE: 0.5019646557047963 - sumary: Puerto Iguazú incauta 1.000 termos. La policía argentina desplega un operativo estratégico con el objetivo de combatirel contrabando
SCORE: 0.6165540311485529 - sumary: La cocaína de Buenos Aires vuelve a poner en peligro el tráfico de 'capsuleros'. Los narcos han replanteado sus rutas y sus rutas
SCORE: 0.5900405887514353 - sumary: La fiscalía acusa a cinco hombres de secuestrar 423 kilos de cocaína en Salta. El conductor y el acompañante de la banda transportaban la droga desde una avioneta
SCORE: 0.5588353015482426 - sumary: El nieto de Robert De Niro murió debido a una sobredosis. El joven actor, que tenía en su cuerpo restos de fenta

# los sumarios de mayor puntaje que quedaron afuera 

In [16]:
for sum, score in zip(sumary_match, summary_match_score):
    if score > 0.44 and score <= 0.5:
        print(f'SCORE: {score} - sumary: {sum}')

SCORE: 0.4431304782629013 - sumary: “La voluntad no es suficiente para poder mejorar la calidad de vida”. La responsable de la Agencia para la Prevención y Asistencia del Abuso de Sustancias y de las Adicciones, Mabel Dell Orfano, asegura que los problemas de violencia, muchísimos, están atravesados por el consumo
SCORE: 0.4772912571206689 - sumary: Un viejo narco de General Roca encuentra sin vida a un hombre de 60 años. El hombre de 60 años se encontraba cumpliendo prisión domiciliaria cuando fue encontrado sin vida
SCORE: 0.44798512011766434 - sumary: Un adolescente detenido por el crimen de Juan Carlos Cruz, el médico cirujano asesinado por tres delincuentes. El acusado será indagado en las próximas horas
SCORE: 0.453640878200531 - sumary: Detenidos dos mujeres en una vivienda allanada. La actividad delictiva dejó como saldo la detención de siete mujeres
SCORE: 0.46183264441788197 - sumary: La Policía de Goya esclareció la sustracción de tres millones de pesos. El ilícito se produj

Conclusión: se podría fijar como criterio de inclución el promedio de los dos scrores: link y sumario. El caso del taxista que denuncia banda narco, y que aparece con un score grande en el link, y uno no muy grande en el sumario es un buen testigo de que se deben balancear ambos documentos.